In [30]:
# import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

import numpy as np
import pandas as pd

from quantfreedom.data.data_dl import data_download_from_ccxt
from quantfreedom.base import backtest_df_only
from quantfreedom.nb.simulate import simulate_up_to_6_nb
from quantfreedom.evaluators.evaluators import is_below
from quantfreedom.indicators.talib_ind import from_talib
from quantfreedom.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    CandleBody,
)
from quantfreedom.enums.enums import Arrays1dTuple
from quantfreedom.plotting.strat_dashboard import strat_dashboard

np.set_printoptions(formatter={"float_kind": "{:.2f}".format})
pd.options.display.float_format = "{:,.2f}".format

# prices = data_download_from_ccxt(
#     exchange='bybit',
#     start='2020-10-01T00:00:00Z',
#     end='2021-04-15T00:00:00Z',
#     symbols=['BTCUSDT', 'ETHUSD'],
#     timeframe='30m',
# )

# prices.to_hdf("yt_prices.hd5", key='data')
price_data = pd.read_hdf('../data/prices.hd5')
rsi_ind = from_talib(
    func_name="rsi",
    price_data=price_data,
    cart_product=False,
    combos=False,
    timeperiod=[15, 30],
)
entries = is_below(
    want_to_evaluate=rsi_ind,
    user_args=[40, 50],
    candle_ohlc="close",
)

In [31]:
Arrays1dTuple._fields

('leverage',
 'max_equity_risk_pct',
 'max_equity_risk_value',
 'risk_rewards',
 'size_pct',
 'size_value',
 'sl_based_on_add_pct',
 'sl_based_on',
 'sl_pcts',
 'sl_to_be_based_on',
 'sl_to_be_trail_by_when_pct_from_avg_entry',
 'sl_to_be_when_pct_from_avg_entry',
 'sl_to_be_zero_or_entry',
 'tp_pcts',
 'tsl_based_on',
 'tsl_pcts_init',
 'tsl_trail_by_pct',
 'tsl_when_pct_from_avg_entry')

In [ ]:
testing_array = Arrays1dTuple(
    max_equity_risk_pct=2.0,
    leverage=6.3,
)

In [2]:
strat_results_df, settings_results_df = backtest_df_only(
    price_data=price_data,
    entries=entries,
    equity=1000.0,
    fee_pct=0.06,
    mmr_pct=0.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=np.arange(7, 8.1, 1),
    risk_rewards=4,
    size_pct=1.0,
    # sl_pcts=np.arange(1,3.1,1),
    tsl_true_or_false=True,
    tsl_pcts_init=8,
    tsl_based_on=CandleBody.low,
    tsl_trail_by_pct=8,
    tsl_when_pct_from_avg_entry=np.arange(3, 4.1, 1),
    gains_pct_filter=150,
    upside_filter=0.6,
    divide_records_array_size_by=2,
)

Checking static variables for errors or conflicts.
Turning all variables into arrays.
Checking arrays for errors or conflicts ... the backtest will begin shortly, please hold.
Creating cartesian product ... after this the backtest will start, I promise :).

Starting the backtest now ... and also here are some stats for your backtest.

Total symbols: 5
Total indicator settings per symbol: 4
Total indicator settings to test: 20
Total order settings per symbol: 4
Total order settings to test: 20
Total candles per symbol: 9,408
Total candles to test: 752,640

Total combinations to test: 80


In [ ]:
sm.show_results([5,20,24])

In [70]:
testing = settings_results_df.iloc[:,[5,20,24]].index.to_list()[2:]

In [73]:
thing = pd.DataFrame(settings_results_df.iloc[2:, [5, 20, 24]])
try:
    thing.loc["sl_to_be_based_on"]
    for i in CandleBody._fields:
        thing.loc["sl_to_be_based_on"].replace(i, getattr(CandleBody, i), inplace=True)
except:
    pass
try:
    thing.loc["tsl_based_on"]
    for i in CandleBody._fields:
        thing.loc["tsl_based_on"].replace(i, getattr(CandleBody, i), inplace=True)
except:
    pass

In [74]:
thing

,5,20,24
max_equity_risk_pct,7.00,8.00,8.00
risk_rewards,4.00,4.00,4.00
size_pct,1.00,1.00,1.00
tsl_based_on,2,2,2
tsl_pcts_init,8.00,8.00,8.00
tsl_trail_by_pct,8.00,8.00,8.00
tsl_when_pct_from_avg_entry,4.00,3.00,3.00


In [75]:
counter = 0
empadf = []
for field in Arrays1dTuple._fields:
    if field == testing[counter]:
        # empadf.append(np.asarray(thing.loc[testing[counter]].values))
        empadf.append(np.asarray(thing.loc[testing[counter]].values, dtype=np.float_))
        counter += 1
    else:
        empadf.append(np.nan)
empadf = Arrays1dTuple(
    leverage=empadf[0],
    max_equity_risk_pct=empadf[1],
    max_equity_risk_value=empadf[2],
    risk_rewards=empadf[3],
    size_pct=empadf[4],
    size_value=empadf[5],
    sl_based_on_add_pct=empadf[6],
    sl_based_on=empadf[7],
    sl_pcts=empadf[8],
    sl_to_be_based_on=empadf[9],
    sl_to_be_trail_by_when_pct_from_avg_entry=empadf[10],
    sl_to_be_when_pct_from_avg_entry=empadf[11],
    sl_to_be_zero_or_entry=empadf[12],
    tp_pcts=empadf[13],
    tsl_based_on=empadf[14],
    tsl_pcts_init=empadf[15],
    tsl_trail_by_pct=empadf[16],
    tsl_when_pct_from_avg_entry=empadf[17],
)
empadf

Arrays1dTuple(leverage=nan, max_equity_risk_pct=array([7.00, 8.00, 8.00]), max_equity_risk_value=nan, risk_rewards=array([4.00, 4.00, 4.00]), size_pct=array([1.00, 1.00, 1.00]), size_value=nan, sl_based_on_add_pct=nan, sl_based_on=nan, sl_pcts=nan, sl_to_be_based_on=nan, sl_to_be_trail_by_when_pct_from_avg_entry=nan, sl_to_be_when_pct_from_avg_entry=nan, sl_to_be_zero_or_entry=nan, tp_pcts=nan, tsl_based_on=array([2.00, 2.00, 2.00]), tsl_pcts_init=array([8.00, 8.00, 8.00]), tsl_trail_by_pct=array([8.00, 8.00, 8.00]), tsl_when_pct_from_avg_entry=array([4.00, 3.00, 3.00]))

In [32]:
Arrays1dTuple._fields

('leverage',
 'max_equity_risk_pct',
 'max_equity_risk_value',
 'risk_rewards',
 'size_pct',
 'size_value',
 'sl_based_on_add_pct',
 'sl_based_on',
 'sl_pcts',
 'sl_to_be_based_on',
 'sl_to_be_trail_by_when_pct_from_avg_entry',
 'sl_to_be_when_pct_from_avg_entry',
 'sl_to_be_zero_or_entry',
 'tp_pcts',
 'tsl_based_on',
 'tsl_pcts_init',
 'tsl_trail_by_pct',
 'tsl_when_pct_from_avg_entry')

In [29]:
price_data[['BTCUSDT', 'ETHUSD', 'ETHUSD']]

symbol                BTCUSDT                                 ETHUSD           \
candle_info              open      high       low     close     open     high   
open_time                                                                       
2020-10-01 00:00:00 10,759.00 10,817.50 10,759.00 10,790.00   359.85   361.45   
2020-10-01 00:30:00 10,790.00 10,790.50 10,775.00 10,777.00   358.95   359.75   
2020-10-01 01:00:00 10,777.00 10,793.00 10,777.00 10,793.00   359.25   359.80   
2020-10-01 01:30:00 10,793.00 10,836.00 10,792.50 10,822.00   359.75   361.40   
2020-10-01 02:00:00 10,822.00 10,841.50 10,808.00 10,808.50   361.20   364.05   
...                       ...       ...       ...       ...      ...      ...   
2021-04-14 21:30:00 62,703.50 62,812.50 62,615.50 62,795.00 2,406.20 2,440.00   
2021-04-14 22:00:00 62,795.00 62,825.00 62,591.00 62,680.00 2,434.05 2,445.00   
2021-04-14 22:30:00 62,680.00 62,993.00 62,680.00 62,849.50 2,427.00 2,438.60   
2021-04-14 23:00:00 62,849.50 62,999.00 62,750.00 62,999.00 2,431.30 2,450.00   
2021-04-14 23:30:00 62,999.00 62,999.00 62,606.00 62,921.00 2,445.95 2,445.95   

symbol                                                                     
candle_info              low    close     open     high      low    close  
open_time                                                                  
2020-10-01 00:00:00   358.85   358.95   359.85   361.45   358.85   358.95  
2020-10-01 00:30:00   358.95   359.25   358.95   359.75   358.95   359.25  
2020-10-01 01:00:00   358.80   359.75   359.25   359.80   358.80   359.75  
2020-10-01 01:30:00   359.75   361.20   359.75   361.40   359.75   361.20  
2020-10-01 02:00:00   361.20   362.95   361.20   364.05   361.20   362.95  
...                      ...      ...      ...      ...      ...      ...  
2021-04-14 21:30:00 2,397.20 2,434.05 2,406.20 2,440.00 2,397.20 2,434.05  
2021-04-14 22:00:00 2,418.65 2,427.00 2,434.05 2,445.00 2,418.65 2,427.00  
2021-04-14 22:30:00 2,423.50 2,431.30 2,427.00 2,438.60 2,423.50 2,431.30  
2021-04-14 23:00:00 2,431.30 2,445.95 2,431.30 2,450.00 2,431.30 2,445.95  
2021-04-14 23:30:00 2,417.40 2,438.20 2,445.95 2,445.95 2,417.40 2,438.20  

[9408 rows x 12 columns]

In [28]:
entries.iloc[:, [1,10,11]]

symbol              BTCUSDT ETHUSD       
RSI_timeperiod           15     30       
RSI_is_below             50     40     50
open_time                                
2020-10-01 00:00:00   False  False  False
2020-10-01 00:30:00   False  False  False
2020-10-01 01:00:00   False  False  False
2020-10-01 01:30:00   False  False  False
2020-10-01 02:00:00   False  False  False
...                     ...    ...    ...
2021-04-14 21:30:00    True  False  False
2021-04-14 22:00:00    True  False  False
2021-04-14 22:30:00    True  False  False
2021-04-14 23:00:00    True  False  False
2021-04-14 23:30:00    True  False  False

[9408 rows x 3 columns]

In [ ]:
order_records = simulate_up_to_6_nb(
    prices=price_data['ETHUSD'].values,
    entries=entries.iloc[:,[11]].values,
    equity=1000.0,
    fee_pct=0.06,
    mmr_pct=0.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=8,
    risk_rewards=3,
    size_pct=1.0,
    # sl_pcts=np.arange(1,3.1,1),
    tsl_true_or_false=True,
    tsl_based_on=CandleBody.low,
    tsl_pcts_init=8,
    tsl_trail_by_pct=8,
    tsl_when_pct_from_avg_entry=3,
    # gains_pct_filter=100,
)

In [ ]:
entries.iloc[:,[11]]

In [ ]:
rsi_ind[[('ETHUSD', 30)]]

In [ ]:
indicator_dict = {
    "indicator1": {
        "values1": rsi_ind[[('ETHUSD', 30)]],
        "entries1": entries.iloc[:,[11]],
    },
}
strat_dashboard(
    indicator_dict=indicator_dict,
    prices=price_data.ETHUSD,
    order_records=order_records,
)